In [92]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import matplotlib
from PIL import Image 
from pylab import *
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import h5py

In [93]:
def get_imlist(path): 
    return [os.path.join(path,f) for f in os.listdir(path) if (f.endswith('.jpg') or f.endswith('.png') or f.endswith('.gif'))]


In [94]:
## img_list = get_imlist("F:\\SP2021\\MCM\MCM_C\\2021MCM_ProblemC_Files\\2021MCM_ProblemC_Files")

all_data = pd.read_csv('./data/Joined.csv', encoding='utf-8', index_col='Detection Date')
# all_data
all_data.index = pd.to_datetime(all_data.index, errors='coerce')
all_data = all_data.sort_values(by = 'Detection Date', ascending=True)

## positive
data_positive = all_data[(all_data['Lab Status'] == "Positive ID")]
positive_img_list = []
x_s, y_s = 250, 250
for file in data_positive['FileName']:
    im = Image.open(os.path.join("F:\\SP2021\\MCM\MCM_C\\2021MCM_ProblemC_Files\\2021MCM_ProblemC_Files", file))
    rgb = im.split()
    red, green, blue = rgb[0], rgb[1], rgb[2]
    r, g, b = rgb[0].resize((x_s, y_s), Image.ANTIALIAS), rgb[1].resize((x_s, y_s), Image.ANTIALIAS), rgb[2].resize((x_s, y_s), Image.ANTIALIAS)
    new_image = Image.merge("RGB", (r, g, b))
    positive_img_list.append(np.array(new_image))

## negative
data_negative = all_data[(all_data['Lab Status'] == "Negative ID") & (all_data['FileType'] == 'image/jpg')]
negative_img_list = []
for file in data_negative['FileName']:
    im = Image.open(os.path.join("F:\\SP2021\\MCM\MCM_C\\2021MCM_ProblemC_Files\\2021MCM_ProblemC_Files", file))
    rgb = im.split()
    red, green, blue = rgb[0], rgb[1], rgb[2]
    r, g, b = rgb[0].resize((x_s, y_s)), rgb[1].resize((x_s, y_s)), rgb[2].resize((x_s, y_s))
    new_image = Image.merge("RGB", (r, g, b))
    negative_img_list.append(np.array(new_image))

Get train/test data Set

In [95]:
## https://blog.csdn.net/u013733326/article/details/79639509
# train_set_x_orig , train_set_y , test_set_x_orig , test_set_y , classes
positive_img_list_added = get_imlist('F:\\SP2021\\MCM2021\\MCM2021_C\\data\\RefImages')
for file in positive_img_list_added:
    im = Image.open(file)
    rgb = im.split()
    red, green, blue = rgb[0], rgb[1], rgb[2]
    r, g, b = rgb[0].resize((x_s, y_s), Image.ANTIALIAS), rgb[1].resize((x_s, y_s), Image.ANTIALIAS), rgb[2].resize((x_s, y_s), Image.ANTIALIAS)
    new_image = Image.merge("RGB", (r, g, b))
    positive_img_list.append(np.array(new_image))

In [122]:
m_train_positive = len(positive_img_list)
m_train_negative = len(negative_img_list)
print('positive:' + str(m_train_positive) + '  negative:' + str(m_train_negative))

train_set_x_orig = np.array(positive_img_list[0:200] + negative_img_list[0:500])
train_set_y = np.array([[1 for i in range(200)] + [0 for i in range(500)]])
train_set_y.reshape(1,700)

test_set_x_orig = np.array(positive_img_list[201:] + negative_img_list[501:551])
test_set_y = np.array([[1 for i in range(10)] + [0 for i in range(50)]])
test_set_y.reshape(1,60)

positive:211  negative:3019


array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [128]:
m_train = train_set_y.shape[1] #训练集里图片的数量。
m_test = test_set_y.shape[1] #测试集里图片的数量。
num_px = train_set_x_orig.shape[1] #训练、测试集里面的图片的宽度和高度（均为64x64）。

#现在看一看我们加载的东西的具体情况
print ("训练集的数量: m_train = " + str(m_train))
print ("测试集的数量 : m_test = " + str(m_test))
print ("每张图片的宽/高 : num_px = " + str(num_px))
print ("每张图片的大小 : (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("训练集_图片的维数 : " + str(train_set_x_orig.shape))
print ("训练集_标签的维数 : " + str(train_set_y.shape))
print ("测试集_图片的维数: " + str(test_set_x_orig.shape))
print ("测试集_标签的维数: " + str(test_set_y.shape))

# index = 6
# plt.imshow(train_set_x_orig[index])
print("y=" + str(train_set_y[:,index]))


训练集的数量: m_train = 700
测试集的数量 : m_test = 60
每张图片的宽/高 : num_px = 250
每张图片的大小 : (250, 250, 3)
训练集_图片的维数 : (700, 250, 250, 3)
训练集_标签的维数 : (1, 700)
测试集_图片的维数: (60, 250, 250, 3)
测试集_标签的维数: (1, 60)


In [130]:
#X_flatten = X.reshape(X.shape [0]，-1).T ＃X.T
train_set_x_flatten  = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T


In [131]:
print ("训练集降维最后的维度： " + str(train_set_x_flatten.shape))
print ("训练集_标签的维数 : " + str(train_set_y.shape))
print ("测试集降维之后的维度: " + str(test_set_x_flatten.shape))
print ("测试集_标签的维数 : " + str(test_set_y.shape))


训练集降维最后的维度： (187500, 700)
训练集_标签的维数 : (1, 700)
测试集降维之后的维度: (187500, 60)
测试集_标签的维数 : (1, 60)


To represent a color image, red, green, and blue channels (RGB) must be specified for each pixel, so the pixel value is actually a vector of three digits in the range 0 to 255.A common preprocessing step in machine learning is to center and normalize the dataset, which means you can subtract the average of the entire NumPy array in each example, and then divide each example by the standard deviation of the entire NumPy array.But for the image dataset, it is simpler and more convenient to divide almost every row of the dataset by 255 (the maximum number of pixel channels). Since there is no data larger than 255 in RGB, we can safely divide by 255 to make the normalized data between 0 and 1. Now we can normalize our dataset:

In [132]:
train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255

In [133]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s


In [134]:
def initialize_with_zeros(dim):
    """
       return 
            w  - vector(dim，1)
            b  - scalar
    """
    w = np.zeros(shape = (dim,1))
    b = 0
    assert(w.shape == (dim, 1)) #w(dim,1)
    assert(isinstance(b, float) or isinstance(b, int)) #b
    return (w , b)


In [161]:
def propagate(w, b, X, Y):
    #     w  - array(num_px * num_px * 3，1)
    #     b  - scalar
    #     X  - array(num_px * num_px * 3，700)
    #     Y  - array(1, 700)

    # return:
    #     cost- cost function
    #     dw  - gradient of w
    #     db  - gradient of b
    m = X.shape[1]
    #forward propagation
    A = sigmoid(np.dot(w.T, X) + b) 
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A))) 
    #bp
    dw = (1 / m) * np.dot(X, (A - Y).T) 
    db = (1 / m) * np.sum(A - Y) 
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw, "db": db}
    return (grads , cost)


In [162]:
#test
print("====================propagate====================")
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))


====================propagate====================
dw = [[0.99993216]
 [1.99980262]]
db = 0.49993523062470574
cost = 6.000064773192205


In [164]:
def optimize(w , b , X , Y , num_iterations , learning_rate , print_cost = False):
    """
    GRADIENT DECENT
    parameter
        w  - (num_px * num_px * 3, 1)
        b  - scalar
        X  - (num_px * num_px * 3, 700)
        Y  - [1/0]
        num_iterations 
        learning_rate 
        print_cost 
    
    return :
        params  - {w, b}
        grads  - {dw, db}
        cost
    """
    costs = []

    for i in range(num_iterations):
        
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db

        if i % 100 == 0:
            costs.append(cost)
        if (print_cost) and (i % 100 == 0):
            print("epoch time: %i, error: %f" % (i,cost))
        
    params  = {
                "w" : w,
                "b" : b }
    grads = {
            "dw": dw,
            "db": db } 
    return (params , grads , costs)


In [166]:
#test optimize
print("====================test optimize====================")
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])
params , grads , costs = optimize(w , b , X , Y , num_iterations=100 , learning_rate = 0.009 , print_cost = False)
print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))


====================test optimize====================
w = [[0.1124579 ]
 [0.23106775]]
b = 1.5593049248448891
dw = [[0.90158428]
 [1.76250842]]
db = 0.4304620716786828


In [167]:
def predict(w , b , X ):   
    m  = X.shape[1] #图片的数量
    Y_prediction = np.zeros((1,m)) 
    w = w.reshape(X.shape[0],1)
    
    #计预测猫在图片中出现的概率
    A = sigmoid(np.dot(w.T , X) + b)
    for i in range(A.shape[1]):
        #将概率a [0，i]转换为实际预测p [0，i]
        Y_prediction[0,i] = 1 if A[0,i] > 0.5 else 0
    #使用断言
    assert(Y_prediction.shape == (1,m))
    
    return Y_prediction


In [168]:
#test predict
print("====================test predict====================")
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])
print("predictions = " + str(predict(w, b, X)))


====================test predict====================
predictions = [[1. 1.]]


In [169]:
def model(X_train , Y_train , X_test , Y_test , num_iterations = 2000 , learning_rate = 0.5 , print_cost = False):
    w , b = initialize_with_zeros(X_train.shape[0])
    
    parameters , grads , costs = optimize(w , b , X_train , Y_train,num_iterations , learning_rate , print_cost)
    
    #从字典“参数”中检索参数w和b
    w , b = parameters["w"] , parameters["b"]
    
    #预测测试/训练集的例子
    Y_prediction_test = predict(w , b, X_test)
    Y_prediction_train = predict(w , b, X_train)
    
    #打印训练后的准确性
    print("训练集准确性："  , format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100) ,"%")
    print("测试集准确性："  , format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100) ,"%")
    
    d = {
            "costs" : costs,
            "Y_prediction_test" : Y_prediction_test,
            "Y_prediciton_train" : Y_prediction_train,
            "w" : w,
            "b" : b,
            "learning_rate" : learning_rate,
            "num_iterations" : num_iterations }
    return d


In [171]:
print("====================test model====================")
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)


====================test model====================
epoch time: 0, error: 0.693147
epoch time: 100, error: nan
epoch time: 200, error: nan
epoch time: 300, error: nan
epoch time: 400, error: nan
epoch time: 500, error: nan
epoch time: 600, error: nan
epoch time: 700, error: nan
epoch time: 800, error: nan
epoch time: 900, error: nan
epoch time: 1000, error: nan
epoch time: 1100, error: nan
epoch time: 1200, error: nan
epoch time: 1300, error: nan
epoch time: 1400, error: nan
epoch time: 1500, error: nan
epoch time: 1600, error: nan
epoch time: 1700, error: nan
epoch time: 1800, error: nan
epoch time: 1900, error: nan
训练集准确性： 100.0 %
测试集准确性： 75.0 %
